In [1]:
import pymysql
import ipywidgets as widgets
import pandas as pd
import glob
import os
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import functools

In [11]:

user_input = widgets.Textarea(
    placeholder='Username',
    description='Username:',
    disabled=False
)
password_input = widgets.Textarea(
    placeholder='Password',
    description='Password:',
    disabled=False
)
host_input = widgets.Textarea(
    placeholder='Host',
    description='Host:',
    disabled=False
)
database_input = widgets.Textarea(
    placeholder='Database',
    description='Database:',
    disabled=False
)
display(user_input, password_input, host_input, database_input)

Textarea(value='', description='Username:', placeholder='Username')

Textarea(value='', description='Password:', placeholder='Password')

Textarea(value='', description='Host:', placeholder='Host')

Textarea(value='', description='Database:', placeholder='Database')

In [12]:
username1 = user_input.value
passwordtxt = password_input.value
hosttxt = host_input.value
databasetxt = database_input.value
print(username1)
print(passwordtxt)
print(hosttxt)
print(databasetxt)
def get_db():
    return pymysql.connect(host=hosttxt,
                             user=username1,
                             password=passwordtxt,
                             database=databasetxt,
                             cursorclass=pymysql.cursors.DictCursor)

In [20]:
# Specify the file path for Excel export
path_input = widgets.Textarea(
    placeholder="C:/Users/AidanO'Gorman/OneDrive - Universal Matter/Shared Documents/Technology/Prod Dev - TG Processing/Milling/AttritionMillData.xlsx",
    description='File Path:',
    disabled=False
)
htmldisplay = widgets.HTML(
    value="C:/Users/AidanO'Gorman/OneDrive - Universal Matter/Shared Documents/Technology/Prod Dev - TG Processing/Milling/AttritionMillData.xlsx",
    placeholder='Some HTML',
)
maxrow_input = widgets.IntSlider(
    value=10,
    min=0,
    max=50,
    step=1,
    description='Max Rows:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
maxcol_input = widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=1,
    description='Max Cols:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(htmldisplay, path_input, maxrow_input, maxcol_input)
# Export to Excel
#with pd.ExcelWriter(excel_file_path) as writer:
#    sort_df.to_excel(writer, sheet_name='Tests', index=False)
#    files_df.to_excel(writer, sheet_name='Files', index=False)

HTML(value="C:/Users/AidanO'Gorman/OneDrive - Universal Matter/Shared Documents/Technology/Prod Dev - TG Proce…

Textarea(value='', description='File Path:', placeholder="C:/Users/AidanO'Gorman/OneDrive - Universal Matter/S…

IntSlider(value=10, continuous_update=False, description='Max Rows:', max=50)

IntSlider(value=30, continuous_update=False, description='Max Cols:')

In [130]:
excel_file_path = path_input.value
pd.set_option('display.max_rows', maxrow_input.value)
pd.set_option('display.max_columns', maxcol_input.value)
# Define the available material types and their corresponding batch information

# Create widgets
batchMultiple_input = widgets.Textarea(
    value='fp-1, f-1, gp-1, d-1',
    placeholder='Input multiple batches separated by a comma',
    description='Batch IDs:',
    disabled=False
)
output_box = widgets.Output()



# Create interactive elements

fetch_button = widgets.Button(
    description='Fetch Data', button_style='info'
)
TreeCheck = widgets.ToggleButton(
    value=False,
    description='Include related tree samples',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
FileCheck = widgets.ToggleButton(
    value=False,
    description='Include all files',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
TestCheck = widgets.ToggleButton(
    value=False,
    description='Include all tests',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
ExportCheck = widgets.ToggleButton(
    value=False,
    description='Export Data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
# Define functions to handle interaction
def fetch_data(material_type, batch_id):
    connection = get_db()
    with output_box:
        clear_output()
        ids = batch_id
        with connection.cursor() as cursor:
            match material_type:
                case "gp":
                    sql = "SELECT * FROM Graphene.FullViewGrapheneBatch WHERE ID=%s"
                case "fp":
                    sql = "SELECT * FROM Graphene.FullViewFeedstockInternalBatch WHERE ID=%s"
                case "f":
                    sql = "SELECT * FROM Graphene.FullViewFlashProcess WHERE ID=%s"
                case "d":
                    sql = "SELECT * FROM Graphene.FullViewDispersionBatch WHERE ID=%s"
                case "fv":
                    sql = "SELECT * FROM Graphene.FullViewVendorBatch WHERE ID=%s" #not made yet I dont think
            cursor.execute(sql, ids)  # Use the loop variable 'ids' here
            result = cursor.fetchall()
        df = pd.DataFrame(result)
        
        # Display the combined DataFrame.
        print(f"Material Type: {material_type}")
        print(f"Batch ID: {batch_id}")
        print("Batch Information:")
        display(df)
    return(df)
        
def fetch_dataMultiple(sample_list):
    with output_box:
        dataframe_list = []
        print(sample_list)
        for samples in sample_list:
            material_type = samples[0]
            batch_id = samples[1]
            ids = batch_id
            with connection.cursor() as cursor:
                match material_type:
                    case "gp":
                        sql = "SELECT * FROM Graphene.FullViewGrapheneBatch WHERE ID=%s"
                    case "fp":
                        sql = "SELECT * FROM Graphene.FullViewFeedstockInternalBatch WHERE ID=%s"
                    case "f":
                        sql = "SELECT * FROM Graphene.FullViewFlashProcess WHERE ID=%s"
                    case "d":
                        sql = "SELECT * FROM Graphene.FullViewDispersionBatch WHERE ID=%s"
                    case "fv":
                        sql = "SELECT * FROM Graphene.FullViewFeedstockInternalBatch WHERE ID=%s" #not made yet I dont think
                cursor.execute(sql, ids)  # Use the loop variable 'ids' here
                result = cursor.fetchall()
            df = pd.DataFrame(result)
            df['Material Type'] = material_type
            dataframe_list.append(df)
    # Display the combined DataFrame.
    dataM_df = pd.concat(dataframe_list, ignore_index=True)
    return dataM_df

def fetch_button_clicked(b):
    fetch_data(material_type_dropdown.value, batch_id_input.value)

def fetchTree_button_clicked(batchID):
    #lookup sample
    material_type, idnumber = batchID.split("-")
    with connection.cursor() as cursor:
        match material_type:
            case "gp":
                sql = "SELECT trees FROM Graphene.FullViewGrapheneBatch WHERE ID=%s"
            case "fp":
                sql = "SELECT trees FROM Graphene.FullViewFeedstockInternalBatch WHERE ID=%s"
            case "f":
                sql = "SELECT trees FROM Graphene.FullViewFlashProcess WHERE ID=%s"
            case "d":
                sql = "SELECT trees FROM Graphene.FullViewDispersionBatch WHERE ID=%s"
            case "fv":
                sql = "SELECT trees FROM Graphene.FullViewVendorBatch WHERE ID=%s" #not made yet I dont think
        cursor.execute(sql, idnumber)  # Use the loop variable 'ids' here
        result = cursor.fetchone()['trees']
    print(result)
    samples = generate_sample_list(result)
    return(samples)
    #data_df = fetch_dataMultiple(sample_list)
    #display(data_df)
    #return(data_df)
    
# Function to handle export button click
def fetchData(b):
    with output_box:
        data_string = batchMultiple_input.value
        sample_list = []
        testframe_list = []
        dataFile_list = []
        samples = data_string.split(", ")  # Split by comma and space
        for sample in samples:
            if TreeCheck.value:
                sampleTree_list = fetchTree_button_clicked(sample)
                sample_list.extend(sampleTree_list)
            else:
                material_type, batch_id = sample.split("-")
                sample_list.append((material_type, batch_id))
        sample_list = set(sample_list)
        for samplepair in sample_list:
            material_type = samplepair[0]
            batch_id = samplepair[1]
            sample = str(material_type+"-"+str(batch_id))
            if TestCheck.value:
                with connection.cursor() as cursor:
                    sql = "SELECT * FROM Graphene.analysisView WHERE `sampleID`=CONVERT(%s USING utf8)"
                    cursor.execute(sql, (sample,) )  # Use the loop variable 'ids' here
                    result = cursor.fetchall()
                    df = pd.DataFrame(result)
                    testframe_list.append(df)
            if FileCheck.value:
                techDir = "C:/Users/AidanO'Gorman/OneDrive - Universal Matter/Shared Documents/Technology/Prod Dev - TG Characterization"
                searchstring = os.path.join(techDir,"**",f"*{sample}**")
                print(searchstring)
                fileList = glob.glob(searchstring,recursive=True)
                for filePath in fileList:
                # Split the path into directory and filename
                    directory, filename = os.path.split(filePath)
                    subfolder = directory.rsplit("Characterization", 1)[1]
                    fileData = [sample, subfolder, filename]
                    dataFile_list.append(fileData)
        data_df = fetch_dataMultiple(sample_list)
        if FileCheck.value:
            files_df = pd.DataFrame(dataFile_list, columns=['ID','subfolder','filename'])
            display(files_df)
        if TestCheck.value:
            combined_df = pd.concat(testframe_list, ignore_index=True)
            sort_df = combined_df.sort_values(by=['sampleID','testType'])
            display(sort_df)
        if len(data_df) > 0:
            display(data_df)
            if ExportCheck.value:
                export_path = "C:/Users/AidanO'Gorman/OneDrive - Universal Matter/combined_data.xlsx"
                with pd.ExcelWriter(export_path) as writer:
                    if TestCheck.value:
                        sort_df.to_excel(writer, sheet_name='Tests', index=False)
                    if FileCheck.value:
                        files_df.to_excel(writer, sheet_name='Files', index=False)
                    data_df.to_excel(writer, sheet_name='Batch results', index=False)
                print(f"Data exported to {export_path}")
        else:
            print("No data to export.")
        
def generate_sample_list(input_string):
    sample_list = []
    
    segments = input_string.split(" / ")
    for segment in segments:
        parts = segment.split("-")
        if len(parts) == 2:
            material_type, batch_id = parts
            sample_list.append((material_type, batch_id))
    
    return sample_list

# Attach the export function to the button's click event
fetch_button.on_click(fetchData)

# Display widgets and instructions
instructions = """
### Interactive Batch Data Viewer

This notebook allows you to select a material type and a batch ID to retrieve relevant batch information.
Simply select the material type from the dropdown, enter the batch ID, and click the 'Fetch Data' button to view the details.
"""

display(widgets.HTML(instructions))
display(batchMultiple_input, TreeCheck, FileCheck, TestCheck, ExportCheck, fetch_button, output_box)

# Create a DataFrame
#df = pd.DataFrame(data)

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

# Display the DataFrame

HTML(value="\n### Interactive Batch Data Viewer\n\nThis notebook allows you to select a material type and a ba…

Textarea(value='fp-1, f-1, gp-1, d-1', description='Batch IDs:', placeholder='Input multiple batches separated…

ToggleButton(value=False, description='Include related tree samples', icon='check', tooltip='Description')

ToggleButton(value=False, description='Include all files', icon='check', tooltip='Description')

ToggleButton(value=False, description='Include all tests', icon='check', tooltip='Description')

ToggleButton(value=False, description='Export Data', icon='check', tooltip='Description')

Button(button_style='info', description='Fetch Data', style=ButtonStyle())

Output()